In [1]:
from copy import deepcopy
import os
import re
import gsd.hoomd
import numpy as np
from morphct import execute_qcc as eqcc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict
from morphct.mobility_kmc import snap_molecule_indices
from morphct.system import System
import pickle

In [2]:
gsd_file = "itic-trajectory.gsd"
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    snap = f[-1]

In [3]:
box = snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = snap.particles.position + snap.particles.image * box
snap.particles.position *= ref_distance
snap.configuration.box[:3] *= ref_distance
unwrap_snap = deepcopy(snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in snap.particles.types]

In [4]:
gsd_mol_index = snap_molecule_indices(snap)
k = np.count_nonzero(gsd_mol_index==0)
chromo_ids = np.arange(snap.particles.N)[0:k]

In [5]:
master_list = []
sublist = chromo_ids
for i in range(len(np.unique(gsd_mol_index))):         
    master_list.append(sublist)
    sublist = [x + k for x in sublist]

In [6]:
%%time
system = System(gsd_file, "itic", frame=-1, scale=3.5636, conversion_dict=amber_dict)
system.add_chromophores(master_list,"acceptor")
system.compute_energies()
system.set_energies()

There are 20 chromophore pairs
Starting singles energy calculation...
Finished in 10.18 s. Output written to itic/singles_energies.txt.
Setting dimer pairs...
Finished in 0.00 s. Output written to itic/dimer_energies.txt.
Energies set.
CPU times: user 8.28 s, sys: 587 ms, total: 8.87 s
Wall time: 18.3 s


In [7]:
lifetime = [1e-10,1e-9]
temp = 300
system.run_kmc(lifetime, temp, n_elec =1000)

---------- KMC_ANALYZE ----------
All figures saved in itic/kmc/figures
---------------------------------
Considering the transport of electron...
Obtaining mean squared displacements...
Plotting distribution of electron displacements
	Figure saved as electron_displacement_dist.png
Calculating mobility...
	Standard Error 0.0
	Fitting r_val = 0.9999999999999998
	Figure saved as lin_MSD_electron.png
	Figure saved as semi_log_MSD_electron.png
	Figure saved as log_MSD_electron.png
	----------------------------------------
	Electron mobility = 2.72E-01  +/- 1.90E-04 cm^2 V^-1 s^-1
	----------------------------------------
Calculating electron trajectory anisotropy...
	----------------------------------------
	Electron charge transport anisotropy: 0.630
	----------------------------------------
Plotting electron hop frequency distribution...
	DYNAMIC CUT
	Notice: No minima found in distribution. Cutoff set to None.
	Cluster cut-off based on hop frequency set to None
	Figure saved as total_ho